In [ ]:
import cv2
import numpy as np
import time
import threading
import atexit
import ipywidgets as widgets
from IPython.display import display

# ==============================================================================
# 1. FastCamera Class
# A threaded class to read frames from the GStreamer pipeline without blocking
# the main processing loop, ensuring a high and stable frame rate.
# ==============================================================================
class FastCamera:
    def __init__(self, width=640, height=480, framerate=30, sensor_id=0):
        # GStreamer pipeline for efficient camera access on Jetson devices.
        # flip-method=2 rotates the image 180 degrees.
        self.gstreamer_pipeline = (
            f"nvarguscamerasrc sensor-id={sensor_id} ! "
            f"video/x-raw(memory:NVMM), width=(int)1280, height=(int)720, framerate=(fraction){framerate}/1 ! "
            f"nvvidconv flip-method=2 ! "
            f"video/x-raw, width=(int){width}, height=(int){height}, format=(string)BGRx ! "
            f"videoconvert ! "
            f"video/x-raw, format=(string)BGR ! appsink drop=true"
        )
        self.cap = cv2.VideoCapture(self.gstreamer_pipeline, cv2.CAP_GSTREAMER)
        if not self.cap.isOpened(): raise RuntimeError("Camera GStreamer failed to open.")
        
        self.frame = np.zeros((height, width, 3), dtype=np.uint8)
        self.running = True
        self.thread = threading.Thread(target=self._reader)
        self.thread.daemon = True
        self.thread.start()
        atexit.register(self.stop) # Ensure camera is released on program exit.

    def _reader(self):
        # Internal method that runs in a separate thread to continuously read frames.
        while self.running:
            ret, frame = self.cap.read()
            if ret: self.frame = frame
        self.cap.release()

    def read(self):
        # Returns the most recently captured frame.
        return self.frame

    def stop(self):
        # Stops the reading thread and releases the camera.
        self.running = False
        if self.thread.is_alive(): self.thread.join()

# ==============================================================================
# 2. Initial Setup
# ==============================================================================
WIDTH, HEIGHT = 480, 270
camera = FastCamera(width=WIDTH, height=HEIGHT)
print("Jetson: Camera Initialized.")

# ==============================================================================
# 3. HSV Color Ranges
# IMPORTANT: These values may need tuning in the competition environment
# based on the specific lighting conditions.
# ==============================================================================
# Red color range (split into two parts as it wraps around 180 in HSV).
lower_red1 = np.array([0, 100, 70])
upper_red1 = np.array([7, 255, 255])
lower_red2 = np.array([160, 65, 65])
upper_red2 = np.array([180, 255, 255])

# Green color range.
lower_green = np.array([40, 30, 75])
upper_green = np.array([90, 255, 255])

# Black color range (defined by a low 'Value' in HSV).
lower_black = np.array([0, 0, 0])
upper_black = np.array([180, 255, 120]) # The upper 'Value' (120) controls sensitivity to darkness.

# ==============================================================================
# 4. Jupyter Notebook Widget Setup
# Creates an image widget to display the combined camera view in the notebook.
# ==============================================================================
# The widget dimensions are doubled to accommodate the 2x2 grid.
combined_widget = widgets.Image(format='jpeg', width=WIDTH*2, height=HEIGHT*2)
display(combined_widget)
print("Jetson: Display widget created. Starting main loop...")
print("To stop, Interrupt the kernel from the top menu.")

# ==============================================================================
# 5. Main Loop
# ==============================================================================
try:
    kernel = np.ones((3, 3), np.uint8) # Kernel for morphological operations (noise removal).
    while True:
        # Read a frame and convert it to HSV color space.
        frame = camera.read()
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Create binary masks for each color.
        red_mask = cv2.add(cv2.inRange(hsv_frame, lower_red1, upper_red1), cv2.inRange(hsv_frame, lower_red2, upper_red2))
        green_mask = cv2.inRange(hsv_frame, lower_green, upper_green)
        black_mask = cv2.inRange(hsv_frame, lower_black, upper_black)

        # Apply a simple morphological 'open' operation to remove small noise from the masks.
        red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, kernel)
        green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel)
        black_mask = cv2.morphologyEx(black_mask, cv2.MORPH_OPEN, kernel)

        # --- Create a 2x2 combined view for display ---
        # 1. Convert the single-channel (grayscale) masks back to three-channel (BGR) images.
        red_mask_bgr = cv2.cvtColor(red_mask, cv2.COLOR_GRAY2BGR)
        green_mask_bgr = cv2.cvtColor(green_mask, cv2.COLOR_GRAY2BGR)
        black_mask_bgr = cv2.cvtColor(black_mask, cv2.COLOR_GRAY2BGR)

        # 2. Stack the images horizontally to create two rows.
        # Top row: Raw Frame | Black Mask
        top_row = np.hstack([frame, black_mask_bgr])
        # Bottom row: Red Mask | Green Mask
        bottom_row = np.hstack([red_mask_bgr, green_mask_bgr])

        # 3. Stack the two rows vertically to create the final 2x2 grid.
        combined_view = np.vstack([top_row, bottom_row])

        # Update the Jupyter widget with the new combined frame.
        _, encoded_frame = cv2.imencode('.jpeg', combined_view)
        combined_widget.value = encoded_frame.tobytes()

        time.sleep(0.05) # Small delay to control the loop speed.

except KeyboardInterrupt:
    print("Loop stopped by user.")
finally:
    # Safely stop the camera thread when the program ends.
    camera.stop()
    print("Jetson program stopped.")